# 0. Setting

## A.패키지 설치

In [4]:
!pip install langchain openai langchain_upstage langchain_core langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.8 MB/s eta 0:

## B. API 키 및 환경 설정

In [3]:
import os
import getpass

# API key 입력
if "UPSTAGE_API_KEY" not in os.environ:
    os.environ["UPSTAGE_API_KEY"] = getpass.getpass("Enter your Upstage API key: ")

# 한글 텍스트를 다루기 위해 인코딩 환경 변수 설정
os.environ["PYTHONIOENCODING"] = "utf-8"

Enter your Upstage API key: ··········


## C. 데이터 로드
구글 드라이브 환경(마운트 필요)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from langchain_upstage import UpstageEmbeddings

# 법률 용어, 위험 조항 사례 데이터
terms_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LLM/web_terms.csv")
clauses_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/LLM/조항.csv")

# 임대차 계약 건(clauseField:5)에 대한 데이터 필터링
data = clauses_df.copy()[clauses_df.clauseField == 5]

# 법률 PDF 파일 경로 지정
pdf_files = {
    "근로기준법": "/content/drive/MyDrive/Colab Notebooks/LLM/근로기준법(법률)(제18176호)(20211119).pdf",
    "대리점거래의 공정화에 관한 법률": "/content/drive/MyDrive/Colab Notebooks/LLM/대리점거래의 공정화에 관한 법률(법률)(제20239호)(20240807).pdf",
    "독점규제 및 공정거래에 관한 법률": "/content/drive/MyDrive/Colab Notebooks/LLM/독점규제 및 공정거래에 관한 법률(법률)(제20239호)(20240807).pdf",
    "민법":"/content/drive/MyDrive/Colab Notebooks/LLM/민법(법률)(제19409호)(20240517).pdf",
    "상가건물 임대차보호법":"/content/drive/MyDrive/Colab Notebooks/LLM/상가건물 임대차보호법(법률)(제18675호)(20220104).pdf",
    "상법":"/content/drive/MyDrive/Colab Notebooks/LLM/상법(법률)(제17764호)(20201229).pdf",
    "소비자기본법":"/content/drive/MyDrive/Colab Notebooks/LLM/소비자기본법(법률)(제20301호)(20240814).pdf",
    "약관의 규제에 관한 법률":"/content/drive/MyDrive/Colab Notebooks/LLM/약관의 규제에 관한 법률(법률)(제20239호)(20240807).pdf",
    "여신전문금융업법":"/content/drive/MyDrive/Colab Notebooks/LLM/여신전문금융업법(법률)(제19260호)(20230622).pdf"
}


## D. Vector Stroe 생성 및 로드

### 법률 데이터 DB

In [ ]:
from transformers import AutoTokenizer
from langchain.text_splitter import TokenTextSplitter
from langchain_upstage import UpstageEmbeddings
from langchain_upstage import UpstageDocumentParseLoader
from langchain.vectorstores import Chroma

# 임베딩 모델
embeddings = UpstageEmbeddings(model="solar-embedding-1-large-passage")

# 토크나이저 및 텍스트 스플리터 로드
solar_tokenizer = AutoTokenizer.from_pretrained("upstage/SOLAR-10.7B-Instruct-v1.0")
token_splitter = TokenTextSplitter.from_huggingface_tokenizer(
    solar_tokenizer, chunk_size=1000, chunk_overlap=200)

# ChormaDB 생성
persist_directory = "/content/drive/MyDrive/Colab Notebooks/LLM/chroma_db" # 저장경로
db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

## 법률 Document 저장(최초 1회; 재실행X)
# docs = []
# for file, path in pdf_files.items():
#   loader = UpstageDocumentParseLoader(path, ocr=False, coordinates = False) # 경로에 있는 pdf 파일 파싱
#   doc = loader.load()
#   doc[0].metadata = {"source":file} # 메타데이터에 파일명(법률명) 저장
#   docs.append(doc[0])

# splits = token_splitter.split_documents(docs) # 텍스트 분할
# db.add_documents(splits) # 분할된 텍스트 DB에 추가
# db.persist() # DB를 경로에 저장

# 벡터스토어 로드
db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)
# VectorStoreRetriever 객체 초기화
retriever = db.as_retriever()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

<ipython-input-5-8fbd15094996>:17: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  db = Chroma(embedding_function=embeddings, persist_directory=persist_directory)


### 위험 조항 사례 데이터

In [ ]:
os.chmod(persist_directory, 0o777)

In [ ]:
from langchain.schema import Document

# 위험 조항 Document 저장(최초 1회; 재실행X)
documents = []
for index, row in data.iterrows():
    sentence = row['clauseArticle'].strip()
    documents.append(Document(page_content=sentence, metadata={"illdcssBasiss": row['illdcssBasiss'],
                                                                 "relateLaword": row['relateLaword']}))
      # content: 위험조항(clauseArticle)/ metadata: 전문가 해석(illdcssBasiss), 법적 근거(relateLaword)

# ChormaDB 생성
persist_directory = "/content/drive/MyDrive/Colab Notebooks/LLM/chroma_data" # 저장경로
vector_store = Chroma.from_documents(documents=documents,
                                     embedding=embeddings,
                                     persist_directory=persist_directory
                                     )

vector_store.persist() # DB를 경로에 저장

# 벡터스토어 로드
vector_store = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
    )

<ipython-input-7-95f4bd6b8ad2>:18: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_store.persist() # DB를 경로에 저장


# 1. OCR

In [10]:
import re
from langchain_upstage import ChatUpstage
from langchain_core.messages import HumanMessage
from flask import Flask, request, jsonify
import requests

import os
import getpass

In [6]:
# OCR API 호출을 통한 문서 텍스트 추출 함수
def extract_text_from_document(UPSTAGE_API_KEY, filename):
    url = "https://api.upstage.ai/v1/document-ai/ocr"
    headers = {"Authorization": f"Bearer {UPSTAGE_API_KEY}"}
    files = {"document": open(filename, "rb")}
    response = requests.post(url, headers=headers, files=files)
    return response.json()

In [11]:
# OCR 함수 호출
filename = "/content/drive/MyDrive/24-2 FALL LLM/부동산 계약서_예시_1.png"

# 환경 변수에서 API 키를 가져옴
api_key = os.environ["UPSTAGE_API_KEY"]

# OCR 함수 호출 시 api_key 전달
ocr_result = extract_text_from_document(api_key, filename)

In [12]:
# ocr_result에서 text만 분리하여 ocr_text로 변경하는 함수
def extract_ocr_text(ocr_result):
    # ocr_result의 'pages' 필드에서 각 페이지의 'text' 필드만 추출
    ocr_text = " ".join(page['text'] for page in ocr_result['pages'])
    return ocr_text

# ocr_text 추출
ocr_text = extract_ocr_text(ocr_result)

In [13]:
# OCR 텍스트에서 '제 x 조' 패턴으로 시작하고, 문장부호로 끝나는 조항을 인식하여 따로 저장하는 함수
def extract_clauses_with_order(ocr_text):
    clauses = []
    processed_text = []

    # "제 x 조" 패턴으로 시작하고, 마침표로 끝나는 조항을 모두 인식
    clause_pattern = re.compile(r'(제\s?\d+\s?조[^.!?]*[.!?])', re.DOTALL)  # 조항이 중간에 줄바꿈이 있어도 인식되도록 DOTALL 플래그 사용
    matches = clause_pattern.finditer(ocr_text)

    last_index = 0
    for match in matches:
        start, end = match.span()
        # 조항 전의 텍스트를 처리
        pre_text = ocr_text[last_index:start].strip()
        if pre_text:
            processed_text.append(pre_text)
        # 조항을 인식하고 저장
        clauses.append(match.group().strip())
        processed_text.append(f"조항: {match.group().strip()}")
        last_index = end

    # 남은 텍스트 처리
    remaining_text = ocr_text[last_index:].strip()
    if remaining_text:
        processed_text.append(remaining_text)

    return clauses, processed_text

# 불필요한 줄바꿈을 제거하는 함수
def clean_text(text):
    return text.replace('\n', ' ').strip()

# 나머지 텍스트에 대해서 조항이 아닌 부분은 전부 '해당없음'으로 처리하는 함수
def classify_remaining_text(remaining_text_list):
    results = []

    # 남은 텍스트를 줄바꿈 기준으로 나누어 처리
    for line in remaining_text_list:
        if line.startswith("조항:"):
            results.append(line)  # 조항은 그대로 처리
        else:
            # 남은 텍스트의 각 줄에 대해 '해당없음' 처리
            lines = line.split("\n")
            for l in lines:
                l = l.strip()
                if l:
                    results.append(f"해당없음: {clean_text(l)}")

    return results

# 최종적으로 조항을 분리하고 나머지를 처리하는 함수, 결과를 딕셔너리로 저장
def process_ocr_text(ocr_text):
    # 조항을 추출하고 순서를 유지하여 텍스트를 처리
    clauses, remaining_text_list = extract_clauses_with_order(ocr_text)

    # 나머지 텍스트에 대해 전부 '해당없음' 처리
    classified_remaining_text = classify_remaining_text(remaining_text_list)

    # 딕셔너리에 저장 (조항과 해당없음)
    final_dict = {}
    for idx, entry in enumerate(classified_remaining_text):
        if entry.startswith("조항:"):
            final_dict[f"item_{idx+1}"] = {"type": "조항", "content": entry.split(": ", 1)[1]}
        else:
            final_dict[f"item_{idx+1}"] = {"type": "해당없음", "content": entry.split(": ", 1)[1]}

    return final_dict

final_classified_text = process_ocr_text(ocr_text)

# 2. Detecton

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 조항이 입력되면 해당 조항이 위험 조항인지 아닌지 분류하는 함수
# 임계치 이상의 유사도를 가지는 조항이 위험 조항 DB에 있을 때
# 그 유사 조항과 전문가 해석, 법적 근거를 출력한다
def detection(clause,threshold = 0.8):
  # 주어진 조항과 가장 유사한 위험 조항 문서를 불러옴
  results = vector_store.similarity_search(clause,k=1)[0]
  sim_clause = results.page_content # 유사 조항
  query_vector = embeddings.embed_query(clause) # 주어진 조항의 벡터
  stored_vector = embeddings.embed_query(sim_clause) # 유사 조항의 벡터

  # 두 조항 간의 코사인 유사도
  cosine_sim = cosine_similarity([query_vector], [stored_vector])

  # 유사한 조항일 경우에만 위험 조항으로 감지하고 정보를 출력함
  if cosine_sim > threshold:
      judgment = results.metadata['illdcssBasiss']
      reason = results.metadata['relateLaword']
      return sim_clause, judgment, reason, 1
  else: return None, None, None, 0

# 3. Explain

## A. 용어 추출 및 설명

In [ ]:
# 해당 조항에서 용어 DB에 존재하는 용어를 추출
def extract_legal_terms(clause, terms_df):
    terms_in_clause = []
    for term in terms_df['term']:
        if term in clause:
            terms_in_clause.append(term)
    return terms_in_clause

# 용어 리스트에 대한 설명을 짝짓는 딕셔너리 생성
def legal_explanations(terms, terms_df):
    explanations = {}
    for term in terms:
        explanation = terms_df[terms_df['term'] == term]['definition'].values
        if explanation:
            explanations[term] = explanation[0]
    return explanations

## B. 조항 설명

### 프롬프트

In [ ]:
from langchain.prompts import PromptTemplate

# 조항 설명 프롬프트
## 위험 조항이 아닐 때 (standard zero-shot prompt)
explanation_template0 = """
  주어진 조항: "{clause}"

  용어 설명: {term_explanations}

  용어 설명을 이용해서, 주어진 조항을 일반인도 이해하기 쉽도록 설명해.
  """
explanation_prompt0 = PromptTemplate(template=explanation_template0, input_variables=["clause", "term_explanations"])

## 위험 조항일 때 (standard 2-shot prompt)
explanation_template1 = """
  주어진 조항은 불리한 조항으로 감지된 조항이다.
  유사 과거 조항에 대한 해석을 바탕으로 주어진 조항이 불합리한 이유를 쉽게 설명해.

  <예시1>
  주어진 조항: "제10항\n입주자는 계약기간을 종료하기전 다음 세입자를 선정해서 임대를 인계해야하고 다음 입주자의 임대보증금을 회사에 입금한 후 환불받는다."

  용어 설명: '계': '상부 상조 친목 공동이익 등을 목적으로 만들어진 한국의 전통 협동조직으로 모임',
 '계약': '법률효과 발생을 목적으로 한 두 개의 의사표시가 합치함으로써 성립하는 하나의 법률행위이다',
 '기간': '일정한 시점에서 다른 시점까지의 시간적인 간격을 의미한다 기간은 그것만으로는 법률요건이 성립되지 않으나 기간의 만료에 의하여 중요한 법률효과를 발생시키는 경우가 많다',
 '보증금': '보증금이란 미래에 발생할 수 있는 서로간의 불이익을 막고자 임차인이 미리 임대인에게 지급하는 금전을 의미합니다 예를 들면 전세계약의 전세금은 보증금에 해당합니다',
 '임대': '임대란 계약의 당사자 가운데 한쪽이 상대편에게 부동산 등 물건을 사용하게 하고 상대편은 이에 대하여 일정한 임차료를 지급할 것을 약속하는 계약입니다 계약을 통해서 빌려 주는 사람은 임대인이 되고 빌리는 사람은 임차인이 됩니다',
 '세입자': '세입자는 일정한 세를 내고 남의 건물이나 방 따위를 빌려 쓰는 사람을 말합니다'

  유사 과거 조항: "제2항\n입주자는 계약기간을 종료하기전 다음 세입자를 선정하여 임대를 인계하고 다음 입주자의 임대보증금을 회사에 입금후 환불받는다."

  유사 과거 조항에 대한 해석: 해당 약관조항은 법률의 규정에 의한 고객의 해지권을 배제하거나 그 행사를 제한하는 조항이며, 계약의 해지로 인한 고객의 원상회복의무를 상당한 이유없이 과중하게 부담시키거나 원상회복청구권을 부당하게 포기하도록 하는 조항이다.

  답변: 주택의 입주자는 본 계약이 끝나기 끝나기 전에, 새로운 세입자를 구해서 그 사람에게 주택을 넘겨야 합니다. 새로운 사람이 보증금을 회사에 보내면 자신이 냈던 보증금을 돌려 받는다는 뜻입니다.
  즉 명시된 계약기간이 끝나기 전에 새로운 계약자를 구해야 하고, 보증금을 돌려받는 시점은 새로운 계약자가 보증금을 지불 한 이후라는 점입니다. 이는 세입자 입장에서 불합리한 조항으로 적용될 수 있으며, 여태까지의 인수인계 과정에 차질이 없었는지 확인해 볼 필요가 있습니다.

  <예시2>
  주어진 조항: "제8조(임대차 등기 등)\n제1항 임차인은 주택임대차보호법에 따라 임대주택에 관한 대항력을 갖추기로 한다. 그리고 갑에게 임대주택에 대한 임차권등기, 전세권등기 또는 (근)저당권등기 등을 요구할 수 없다."

  용어 설명: '대항력': '이미 유효하게 발생하고 있는 법률관계를 제자에 대하여 주장할 수 있는 법률상의 효력을 말한다',
 '임대차': '임대인이 임차인에게 어떤 물건을 사용수익하게 할 것을 약정하고 임차인이 이에 대하여 차임을 지급할 것을 약정함으로써 성립하는 계약민법 제조제조',
 '저당권': '채권자가 채무자나 또는 제자가 채무담보로서 제공한 부동산 또는 부동산물권을 인도받지 않고 다만 관념상으로만 지배하여 채무의 변제가 없을 경우 그 목적물로부터 우선변제를 받을 수 있는 권리',
 '전세권': '전세금을 지급하고 타인의 부동산을 일정기간 그 용도에 따라 사용 수익한 후 그 부동산을 반환하고 전세금을 다시 돌려받는 권리민법 제조',
 '등기': '등기란 법정절차에 따라서 부동산의 권리관계를 등기부에 등록하는 행위 또는 기재 그 자체를 의미합니다',
 '임대': '임대란 계약의 당사자 가운데 한쪽이 상대편에게 부동산 등 물건을 사용하게 하고 상대편은 이에 대하여 일정한 임차료를 지급할 것을 약속하는 계약입니다 계약을 통해서 빌려 주는 사람은 임대인이 되고 빌리는 사람은 임차인이 됩니다',
 '임대주택': '임대주택이란 국가 또는 민간 건설업체가 건축하여 주민에게 임대하는 주택입니다',
 '임차': '임차란 돈을 내고 타인의 건물을 빌리는 것을 의미합니다',
 '임차권': '임차권이란 임대차 계약에서 빌려 쓰는 사람이 그 건물을 사용하고 이익을 얻을 수 있는 권리를 의미합니다',
 '임차권등기': '임차권등기란 임대차 계약이 종료됐으나 보증금을 돌려 받지 못한 상태에서 이사를 가야 할 경우에 대항력을 유지하기 위해 설정하는 등기를 의미합니다',
 '임차인': '임차인이란 임대차 계약에서 돈을 내고 건물을 빌려 쓰는 사람입니다',
 '주택임대차보호법': '주택임대차보호법이란 국민 주거생활의 안정을 보장함을 목적으로 주거용 건물의 임대차에 관하여 민법에 대한 특례를 규정한 법률입니다',
 '전세': '전세란 주택이나 건물을 가진 사람에게 일정한 금액을 맡기고 그 주택이나 집을 일정 기간 동안 빌리는 것을 말합니다'

  유사 과거 조항: "제16조(임대차 등기 등)\n제1항 임차인은 주택임대차보호법에 따라 임대주택에 관한 대항력을 갖추기로 하며, 갑에게 임대주택에 관한 임차권등기, 전세권등기, 또는 (근)저당권등기를 요구할 수 없다."

  유사 과거 조항에 대한 해석: 특별한 사유도 없이 일방적으로 임차인이 임대인에게 임대주택에 관한 임대차등기 등을 요구할 수 없도록 규정하고 있는 바, 이는 민법 제621조에서 규정하고 있는 임차인의 법률상 권리를 상당한 이유 없이 배제하고 있다.

  답변: 임차인은 임대주택에 대해 주택임대차보호법에 따른 대항력을 갖추기로 되어 있지만, 본 계약에서는 임차인이 임대인에게 임차권등기, 전세권등기, 또는 (근)저당권등기를 요구할 수 없다고 명시하고 있습니다. 즉, 임차인은 특별한 사유 없이도 임대인에게 이러한 등기를 요구할 수 없다는 뜻입니다.
  이 조항은 임차인의 권리를 상당한 이유 없이 배제하는 것으로 볼 수 있습니다. 민법에서는 임차인이 필요할 때 임차권등기를 통해 자신의 권리를 보호할 수 있도록 하고 있는데, 이 조항은 그 권리를 제한하고 있기 때문입니다. 임차권등기나 전세권등기는 임차인이 주택의 소유권 이전이나 임대인의 재정 문제 등으로부터 안전을 확보할 수 있는 중요한 수단인데, 이 조항은 그런 보호를 받지 못하도록 막고 있습니다.
  따라서, 임차인 입장에서는 이 조항이 불합리할 수 있다는 점을 인지하고, 계약서를 주의 깊게 검토하고 협의해볼 필요가 있습니다.

  <질문>
  주어진 조항: "{clause}"

  용어 설명: {term_explanations}

  유사 과거 조항: "{corr_clause}"

  유사 과거 조항에 대한 해석: {judgment}

  답변:
  """
explanation_prompt1 = PromptTemplate(template=explanation_template1, input_variables=["clause", "term_explanations","corr_ex","judgment"])

### LLM Chain with chat api

In [ ]:
from langchain_upstage import ChatUpstage

# LangChain의 PromptTemplate과 LLM을 결합하여 chain 생성
model = 'solar-1-mini-chat'

def explanation(clause):
  # 위험 조항 감지
  sim_clause, judgment, reason, detect = detection(clause)
  # 법률 용어 설명 저장
  legal_terms = extract_legal_terms(clause, terms_df)
  term_explanations = legal_explanations(legal_terms, terms_df)

  llm = ChatUpstage(model=model)
  if not detect: # 위험 조항이 아닐 때
    chain = explanation_prompt0|llm
    explan = chain.invoke({"clause": clause, "term_explanations":term_explanations}).content
  else: # 위험 조항일 때
    chain = explanation_prompt1|llm
    explan = chain.invoke({"clause": clause, "term_explanations":term_explanations, "corr_clause": sim_clause, "judgment":judgment}).content

  if reason is None:
    return explan, None
  else:
    reason = reason.split('<sep>')
    # RAG를 이용한 법적 근거 추적
    reasons = []
    for r in reason:
      context_docs = retriever.invoke(r)
      r = context_docs[0].metadata['source'] + " " + r
      reasons.append(r)
    return explan, reasons


## C. 법적 근거 추적
데이터에서 제시하는 법률 조항은 어느 법률에서 온 것인지 출처를 알 수 없음.
법률 데이터가 저장된 retriever를 이용해 참조한 문서의 법률 이름을 가져오도록 함

In [ ]:
data.relateLaword.head()

,relateLaword
14,제8조(손해배상액의 예정) 고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무...
33,제8조(손해배상액의 예정) 고객에게 부당하게 과중한 지연 손해금 등의 손해배상 의무...
50,제7조(면책조항의 금지) 계약 당사자의 책임에 관하여 정하고 있는 약관의 내용 중 ...
68,제6조(일반원칙)\n ② 약관의 내용 중 다음 각 호의 어느 하나에 해당하는 내용을...
87,제6조(일반원칙) \n② 약관의 내용 중 다음 각 호의 어느 하나에 해당하는 내용을...


### 예시

In [ ]:
context_docs = retriever.invoke('제8조(손해배상액의 예정) 고객에게 부당하게 과중한..')
context_docs[0].metadata['source'] # 참조한 문서의 메타데이터

'약관의 규제에 관한 법률'

In [ ]:
e, r = explanation("제8조(임대차 등기 등)\n제1항 임차인은 주택임대차보호법에 따라 임대주택에 관한 대항력을 갖추기로 한다. 그리고 갑에게 임대주택에 대한 임차권등기, 전세권등기 또는 (근)저당권등기 등을 요구할 수 없다.")

In [ ]:
r

['약관의 규제에 관한 법률 제11조(고객의 권익 보호) \n고객의 권익에 관하여 정하고 있는 약관의 내용 중 다음 각 호의 어느 하나에 해당하는 내용을 정하고 있는 조항은 무효로 한다.\n1. 법률에 따른 고객의 항변권, 상계권 등의 권리를 상당한 이유 없이 배제하거나 제한하는 조항']